In [1]:
import pandas as pd

In [2]:
# red the csv file containing ethnicities
ethnicity = pd.read_csv('2018-census-place-summaries-ethnicity-table2-2018-csv.csv')

In [3]:
# check how many rows there are
print(len(ethnicity))

21249


In [4]:
# check how many unique values for area_description there are
len(ethnicity['Area_description'].unique())

2359

## Regions

In [5]:
## extract totals for regions

# extract only rows where the totals are included
data_region = ethnicity.loc[ethnicity['Ethnic_group_total_responses_code'] == 'Total']
# exclude totals for the whole dataset
data_region = data_region.loc[data_region['Area_code'] != 'Total']
# change area code to numeric
data_region['Area_code'] = pd.to_numeric(data_region['Area_code'])
# extract only codes smaller than 100 - these are the regions
data_region = data_region.loc[data_region['Area_code'] < 100]
# only keep 2 columns
data_region = data_region[['Area_description', 'Census_usually_resident_population_count']]
# change their name
data_region = data_region.rename(columns={'Area_description': 'STNAME', 'Census_usually_resident_population_count': 'TOT_POP'})
# display sample
data_region.head()

,STNAME,TOT_POP
7,Northland Region,179076
16,Auckland Region,1571718
25,Waikato Region,458202
34,Bay of Plenty Region,308499
43,Gisborne Region,47517


In [6]:
data_region

,STNAME,TOT_POP
7,Northland Region,179076
16,Auckland Region,1571718
25,Waikato Region,458202
34,Bay of Plenty Region,308499
43,Gisborne Region,47517
52,Hawke's Bay Region,166368
61,Taranaki Region,117561
70,Manawatu-Wanganui Region,238797
79,Wellington Region,506814
88,West Coast Region,31575


In [7]:
# there should be 17 unique values - 16 regions plus area outside of regions
data_region['STNAME'].unique()

array(['Northland Region', 'Auckland Region', 'Waikato Region',
       'Bay of Plenty Region', 'Gisborne Region', "Hawke's Bay Region",
       'Taranaki Region', 'Manawatu-Wanganui Region', 'Wellington Region',
       'West Coast Region', 'Canterbury Region', 'Otago Region',
       'Southland Region', 'Tasman Region', 'Nelson Region',
       'Marlborough Region', 'Area Outside Region'], dtype=object)

In [8]:
# remove ' Region' from all names
data_region['STNAME'] = data_region['STNAME'].str.replace(' Region','')
data_region['STNAME'].unique()

array(['Northland', 'Auckland', 'Waikato', 'Bay of Plenty', 'Gisborne',
       "Hawke's Bay", 'Taranaki', 'Manawatu-Wanganui', 'Wellington',
       'West Coast', 'Canterbury', 'Otago', 'Southland', 'Tasman',
       'Nelson', 'Marlborough', 'Area Outside'], dtype=object)

In [9]:
# check the number of records
data_region['TOT_POP'] = pd.to_numeric(data_region['TOT_POP'])
data_region['TOT_POP'].sum()

4699752

## Areas

In [10]:
# extract only rows area type is not a Regional Council
data_area = ethnicity.loc[ethnicity['Area_type'] == 'Statistical Area 2']
# only include totals
data_area = data_area.loc[data_area['Ethnic_group_total_responses_code'] == 'Total']
# exclude totals for the whole dataset
data_area = data_area.loc[data_area['Area_code'] != 'Total']
# only keep 2 columns
data_area = data_area[['Area_description', 'Census_usually_resident_population_count']]
# change their names
data_area = data_area.rename(columns={'Area_description': 'NAME', 'Census_usually_resident_population_count': 'POPESTIMATE2015'})
# display sample
data_area.head()

,NAME,POPESTIMATE2015
169,North Cape,1602
178,Rangaunu Harbour,2310
187,Inlets Far North District,45
196,Karikari Peninsula,1251
205,Tangonge,1134


In [11]:
# check the number of records
data_area['POPESTIMATE2015'] = pd.to_numeric(data_area['POPESTIMATE2015'])
data_area['POPESTIMATE2015'].sum()

4699776

The totals are very similar, there is a difference of 24 records

In [12]:
# count how many unique areas there are
len(data_area['NAME'].unique())

2253

I have 2,253 areas to match to 16 regions.

## Match area to region
As I couldn't find any list that would match areas to regions, I decided to use GeoPy package which can retrieve an address based on a keyword. From that address, I am planning to extract region when possible.

In [13]:
# install needed libraries
!pip install geopy 
!pip install Nominatim

  Created wheel for Nominatim: filename=nominatim-0.1-py3-none-any.whl size=2363 sha256=fe13d868e73e8ab4bab5b26b00b9d4167bf61702579040b9b8d2e548ef480044
  Stored in directory: /root/.cache/pip/wheels/37/00/9e/d904c390bfb174830ad3dcfd62af5544cee7d88bed4f8acedd
Successfully built Nominatim


In [14]:
# set up a list of regions
regions = data_region['STNAME'].unique().tolist()

In [15]:
# replace the name of one of the regions because of an accent
regions[7] = 'Manawatū-Whanganui'

In [16]:
# I can use geopy to try and extract an address based on an area name
# from this address, I can extract the region
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Your_Name")
city ="Kaikoura District"
country ="New Zealand"
loc = geolocator.geocode(city+','+ country)
print(loc.address)

Kaikōura District, Canterbury, New Zealand / Aotearoa


In [17]:
# add a new column for STNAME into the area dataframe
data_area['STNAME'] = ''

In [19]:
# this can take a bit of time (19 mins)

# iterate over all areas
for index, row in data_area.iterrows():
  # if an area doesn't have a specified state
  #if pd.isna(row['STNAME']):
  if row['STNAME'] == '':
    # get the location
    location = geolocator.geocode(row['NAME']+','+ "New Zealand")
    region = ''
    if location != None:
      # iterate over possible regions
      for i in regions:
        # if any of them are part of the address string
        if i in location.address:
          region = i
    # if location cannot be found, try some other possibilities     
    if location == None:
      try:
        area = row['NAME'].split('-')[0]
        location = geolocator.geocode(area+','+ "New Zealand")
        region = ''
        if location != None:
          # iterate over possible regions
          for i in regions:
            # if any of them are part of the address string
            if i in location.address:
              region = i
      except Exception:
        pass

    # add it as a region for that area
    data_area.at[index,'STNAME'] = region

In [20]:
# check how many areas each region has
data_area['STNAME'].value_counts()

Auckland              496
Canterbury            306
Wellington            221
Waikato               219
                      179
Manawatū-Whanganui    139
Otago                 127
Bay of Plenty         126
Northland             105
Hawke's Bay            82
Southland              64
Taranaki               50
West Coast             38
Tasman                 27
Marlborough            26
Gisborne               25
Nelson                 23
Name: STNAME, dtype: int64

In [21]:
# check how many people there are in each region
data_area.groupby(['STNAME']).sum()

,POPESTIMATE2015
STNAME,
,163086
Auckland,1390761
Bay of Plenty,285009
Canterbury,604926
Gisborne,45063
Hawke's Bay,155187
Manawatū-Whanganui,247659
Marlborough,46599
Nelson,42060


179 areas have not been classified into any region. Seem like quite a high number. It also contains 163,086 individuals. I will classify these as `Area outside`

In [24]:
# classify all unclassified as `Area outside`
data_area = data_area.fillna(value='Area Outside')

In [25]:
# get sums for each region
regions = data_area.groupby(['STNAME']).sum()
regions

,POPESTIMATE2015
STNAME,
,163086
Auckland,1390761
Bay of Plenty,285009
Canterbury,604926
Gisborne,45063
Hawke's Bay,155187
Manawatū-Whanganui,247659
Marlborough,46599
Nelson,42060


In [28]:
# assign region population to each area - based on sum
data = pd.merge(data_area, regions, on=["STNAME", "STNAME"])
data = data.rename(columns={'POPESTIMATE2015_x': 'POPESTIMATE2015', 'POPESTIMATE2015_y': 'TOT_POP'})
data

,NAME,POPESTIMATE2015,STNAME,TOT_POP
0,North Cape,1602,Northland,201141
1,Rangaunu Harbour,2310,Northland,201141
2,Karikari Peninsula,1251,Northland,201141
3,Tangonge,1134,Northland,201141
4,Ahipara,1230,Northland,201141
...,...,...,...,...
2248,Grampians,2412,Nelson,42060
2249,Enner Glynn,2916,Nelson,42060
2250,The Brook,1992,Nelson,42060
2251,Nelson Creek,669,Nelson,42060


## Male/Female split

In [29]:
# load the dataset I will use for gender
gender_source = pd.read_csv('2018-census-place-summaries-popdwell-table4-2018-csv.csv')
gender_source.head()

,Year,Area_type,Area_code,Area_description,Age_group_life_cycle_groups_code,Age_group_life_cycle_groups_description,Sex_code,Sex_description,Maori_ethnic_group_indicator_summary_code,Maori_ethnic_group_indicator_summary_description,Census_usually_resident_population_count,Age_group_life_cycle_groups_by_Maori_ethnic_group_indicator_summary_percent,Age_group_life_cycle_groups_by_sex_percent
0,2018,Regional Council,01,Northland Region,1,Under 15 years,1,Male,00,No 'Maori' response given,8622,..,9.7
1,2018,Regional Council,01,Northland Region,1,Under 15 years,1,Male,20,Maori,10557,33.3,11.9
2,2018,Regional Council,01,Northland Region,1,Under 15 years,1,Male,Total,Total,19176,..,21.6
3,2018,Regional Council,01,Northland Region,1,Under 15 years,2,Female,00,No 'Maori' response given,8328,..,9.2
4,2018,Regional Council,01,Northland Region,1,Under 15 years,2,Female,20,Maori,10116,30.9,11.2


In [31]:
# only keep the areas
gender = gender_source.loc[gender_source['Area_type'] == 'Statistical Area 2']
# only keep totals per area
gender = gender.loc[gender['Age_group_life_cycle_groups_code'] == 'Total']
# only keep totals per gender
gender = gender.loc[gender['Maori_ethnic_group_indicator_summary_code'] == 'Total']
# but not total overall
gender = gender.loc[gender['Sex_code'] != 'Total']
# only keep relevant columns
gender = gender[['Area_description', 'Sex_description', 'Census_usually_resident_population_count']]
# only keep numeric values - some areas have no populations
gender = gender[pd.to_numeric(gender['Census_usually_resident_population_count'], errors='coerce').notnull()]
# convert the numeric column to numeric type
gender['Census_usually_resident_population_count'] = pd.to_numeric(gender['Census_usually_resident_population_count'])
# create a pivot table
gender = gender.pivot_table('Census_usually_resident_population_count', ['Area_description'], 'Sex_description')
# check the data
gender.head()

Sex_description,Female,Male
Area_description,,
Abbey Caves-Glenbervie,672,654
Abbotsford,1413,1401
Acacia Bay,825,825
Addington East,1458,1482
Addington North,9,3


In [32]:
# for each area calculate the proportion of Male and Female
# iterate over all areas
for index, row in gender.iterrows():
  # and calculate proportions
  gender.at[index,'TOT_MALE'] = row['Female']/(row['Male'] + row['Female'])
  gender.at[index,'TOT_FEMLE'] = row['Male']/(row['Male'] + row['Female'])


#df.index = df.index.droplevel(1) 
# only keep relevant columns
gender = gender.drop(columns=['Male', 'Female'])

# add name as one of the columns
gender['NAME'] = gender.index

# display the result
gender.head()

Sex_description,TOT_MALE,TOT_FEMLE,NAME
Area_description,,,
Abbey Caves-Glenbervie,0.506787,0.493213,Abbey Caves-Glenbervie
Abbotsford,0.502132,0.497868,Abbotsford
Acacia Bay,0.500000,0.500000,Acacia Bay
Addington East,0.495918,0.504082,Addington East
Addington North,0.750000,0.250000,Addington North


In [33]:
# merge with the rest of the data
data = pd.merge(data, gender, on=["NAME", "NAME"])
data.head()

,NAME,POPESTIMATE2015,STNAME,TOT_POP,TOT_MALE,TOT_FEMLE
0,North Cape,1602,Northland,201141,0.498127,0.501873
1,Rangaunu Harbour,2310,Northland,201141,0.495449,0.504551
2,Karikari Peninsula,1251,Northland,201141,0.497608,0.502392
3,Tangonge,1134,Northland,201141,0.493369,0.506631
4,Ahipara,1230,Northland,201141,0.515815,0.484185


In [34]:
print(len(data))

2154


The merge only kept matched data in both dataframes. Hence areas with population 0 weren't kept and as a result I have 2,154 areas left.

## Ethnicities
Synthea only includes 6 categories for etchnivities - `White`, `Hispanic`, `Black`, `Asian`, `Native` and  `Other`. However, the census data for NZ specifies 7 ethnicities: `European`, `Maori`, `Pacific peoples`, `Asian`, `Middle Eastern/Latin American/African`, `Other` and `Not eslewhere specified`.<br>
I will consider `European` as `White`, `Maori` and `Pacific peoples` as `Native`, `Asian` as `Asian` and all the others as `Other`.

In [35]:
# read the csv file containing ethnicities
ethnicity_source = pd.read_csv('2018-census-place-summaries-ethnicity-table2-2018-csv.csv')
ethnicity_source.head()

,Year,Area_type,Area_code,Area_description,Ethnic_group_total_responses_code,Ethnic_group_total_responses_description,Census_usually_resident_population_count,Ethnic_group_total_responses_percent
0,2018,Regional Council,01,Northland Region,1,European,130971,73.1
1,2018,Regional Council,01,Northland Region,2,Maori,64458,36.0
2,2018,Regional Council,01,Northland Region,3,Pacific peoples,7542,4.2
3,2018,Regional Council,01,Northland Region,4,Asian,7041,3.9
4,2018,Regional Council,01,Northland Region,5,Middle Eastern/Latin American/African,900,0.5


In [36]:
# only keep the areas
ethnicity = ethnicity_source.loc[ethnicity_source['Area_type'] == 'Statistical Area 2']
# remove total
ethnicity = ethnicity.loc[ethnicity['Ethnic_group_total_responses_description'] != 'Total']
# remove total stated
ethnicity = ethnicity.loc[ethnicity['Ethnic_group_total_responses_description'] != 'Total stated']
# only keep relevant columns
ethnicity = ethnicity[['Area_description', 'Ethnic_group_total_responses_description', 'Census_usually_resident_population_count']]
# only keep numeric values - some areas have no populations
ethnicity = ethnicity[pd.to_numeric(ethnicity['Census_usually_resident_population_count'], errors='coerce').notnull()]
# convert the numeric column to numeric type
ethnicity['Census_usually_resident_population_count'] = pd.to_numeric(ethnicity['Census_usually_resident_population_count'])
# create a pivot table
ethnicity = ethnicity.pivot_table('Census_usually_resident_population_count', ['Area_description'], 'Ethnic_group_total_responses_description')
# check the data
ethnicity.head()

Ethnic_group_total_responses_description,Asian,European,Maori,Middle Eastern/Latin American/African,Not elsewhere included,Other ethnicity,Pacific peoples
Area_description,,,,,,,
Abbey Caves-Glenbervie,42.0,1224.0,210.0,3.0,0.0,12.0,21.0
Abbotsford,78.0,2646.0,255.0,6.0,0.0,30.0,45.0
Acacia Bay,30.0,1569.0,162.0,6.0,0.0,24.0,9.0
Addington East,807.0,1818.0,336.0,87.0,0.0,45.0,135.0
Addington North,NaN,9.0,NaN,NaN,NaN,NaN,NaN


In [37]:
# for each area calculate the proportion of each type of etchnicity
# iterate over all areas
for index, row in ethnicity.iterrows():
  # calculate sum total
  total = row['Asian'] + row['European'] + row['Maori'] + \
          row['Middle Eastern/Latin American/African'] + row['Not elsewhere included'] + \
          row['Other ethnicity'] + row['Pacific peoples']
  # and calculate proportions
  ethnicity.at[index,'WHITE'] = row['European']/total
  ethnicity.at[index,'HISPANIC'] = 0
  ethnicity.at[index,'BLACK'] = 0
  ethnicity.at[index,'ASIAN'] = row['Asian']/total
  ethnicity.at[index,'NATIVE'] = (row['Maori'] + row['Pacific peoples'])/total
  ethnicity.at[index,'OTHER'] = (row['Middle Eastern/Latin American/African'] + \
                                 row['Not elsewhere included'] + row['Other ethnicity'])/total

 
# only keep relevant columns
ethnicity = ethnicity.drop(columns=['Asian', 'European', 'Maori', \
                              'Middle Eastern/Latin American/African', \
                              'Not elsewhere included', 'Other ethnicity', \
                              'Pacific peoples'])

# add name as one of the columns
ethnicity['NAME'] = ethnicity.index

# display the result
ethnicity.head()

Ethnic_group_total_responses_description,WHITE,HISPANIC,BLACK,ASIAN,NATIVE,OTHER,NAME
Area_description,,,,,,,
Abbey Caves-Glenbervie,0.809524,0.0,0.0,0.027778,0.152778,0.009921,Abbey Caves-Glenbervie
Abbotsford,0.864706,0.0,0.0,0.025490,0.098039,0.011765,Abbotsford
Acacia Bay,0.871667,0.0,0.0,0.016667,0.095000,0.016667,Acacia Bay
Addington East,0.563197,0.0,0.0,0.250000,0.145911,0.040892,Addington East
Addington North,NaN,0.0,0.0,NaN,NaN,NaN,Addington North


In [38]:
# merge with the rest of the data
data = pd.merge(data, ethnicity, on=["NAME", "NAME"])
data.head()

,NAME,POPESTIMATE2015,STNAME,TOT_POP,TOT_MALE,TOT_FEMLE,WHITE,HISPANIC,BLACK,ASIAN,NATIVE,OTHER
0,North Cape,1602,Northland,201141,0.498127,0.501873,0.450156,0.0,0.0,0.012461,0.526480,0.010903
1,Rangaunu Harbour,2310,Northland,201141,0.495449,0.504551,0.485521,0.0,0.0,0.016409,0.485521,0.012548
2,Karikari Peninsula,1251,Northland,201141,0.497608,0.502392,0.565302,0.0,0.0,0.011696,0.407407,0.015595
3,Tangonge,1134,Northland,201141,0.493369,0.506631,0.483673,0.0,0.0,0.008163,0.491837,0.016327
4,Ahipara,1230,Northland,201141,0.515815,0.484185,0.460259,0.0,0.0,0.014787,0.508318,0.016636


In [39]:
print(len(data))

2153


After this merge, I lost one more area due to insufficient data.

## Age groups

In [40]:
# read the csv file containing age groups
age_source = pd.read_csv('2018-census-place-summaries-popdwell-table3-2018-csv.csv')
age_source.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Year,Area_type,Area_code,Area_description,Age_group_5_year_groups_to_85_years_and_over_code,Age_group_5_year_groups_to_85_years_and_over_description,Sex_code,Sex_description,Maori_ethnic_group_indicator_summary_code,Maori_ethnic_group_indicator_summary_description,Census_usually_resident_population_count,Age_group_5_year_groups_to_85_years_and_over_by_Maori_ethnic_group_indicator_summary_percent,Age_group_5_year_groups_to_85_years_and_over_by_sex_percent
0,2018,Regional Council,01,Northland Region,01,0-4 years,1,Male,00,No 'Maori' response given,2607,..,2.9
1,2018,Regional Council,01,Northland Region,01,0-4 years,1,Male,20,Maori,3333,10.5,3.8
2,2018,Regional Council,01,Northland Region,01,0-4 years,1,Male,Total,Total,5943,..,6.7
3,2018,Regional Council,01,Northland Region,01,0-4 years,2,Female,00,No 'Maori' response given,2490,..,2.8
4,2018,Regional Council,01,Northland Region,01,0-4 years,2,Female,20,Maori,3249,9.9,3.6


In [46]:
# for the purpose of uploading the source files into GitHub, I created a file with
# reduced size that has some of the fields pre-filtered
# the original file can be downloaded from the NZ census website


# only keep the areas
age = age_source.loc[age_source['Area_type'] == 'Statistical Area 2']
# only keep totals per age range
age = age.loc[age['Age_group_5_year_groups_to_85_years_and_over_description'] != 'Total']
age = age.loc[age['Sex_description'] == 'Total']
age = age.loc[age['Maori_ethnic_group_indicator_summary_description'] != ' Total']

age.to_csv('2018-census-place-summaries-popdwell-table3-2018-csv.csv', index=False)

In [47]:
# only keep the areas
age = age_source.loc[age_source['Area_type'] == 'Statistical Area 2']
# only keep totals per age range
age = age.loc[age['Age_group_5_year_groups_to_85_years_and_over_description'] != 'Total']
age = age.loc[age['Sex_description'] == 'Total']
age = age.loc[age['Maori_ethnic_group_indicator_summary_description'] != ' Total']

# only keep relevant columns
age = age[['Area_description', 'Age_group_5_year_groups_to_85_years_and_over_code', 'Census_usually_resident_population_count']]
# only keep numeric values - some areas have no populations
age = age[pd.to_numeric(age['Census_usually_resident_population_count'], errors='coerce').notnull()]
# convert the numeric column to numeric type
age['Census_usually_resident_population_count'] = pd.to_numeric(age['Census_usually_resident_population_count'])
# create a pivot table
age = age.pivot_table('Census_usually_resident_population_count', ['Area_description'], 'Age_group_5_year_groups_to_85_years_and_over_code')
# check the data
age.head()

Age_group_5_year_groups_to_85_years_and_over_code,01,02,03,04,05,06,07,08,09,10,11,12,13,14,15,16,17,18
Area_description,,,,,,,,,,,,,,,,,,
Abbey Caves-Glenbervie,50.0,54.0,70.0,57.0,34.0,26.0,38.0,48.0,57.0,90.0,72.0,96.0,62.0,73.0,73.5,31.5,12.0,6.0
Abbotsford,121.0,137.0,107.0,121.0,88.0,142.0,118.0,140.0,128.0,126.0,138.0,127.0,168.0,121.5,99.0,97.5,55.5,27.0
Acacia Bay,55.0,52.0,70.0,56.0,24.0,58.5,36.0,58.0,61.0,85.0,100.0,99.0,83.0,166.5,135.0,70.5,25.5,18.0
Addington East,127.0,93.0,57.0,80.0,237.0,290.0,227.0,150.0,124.0,114.0,104.0,81.0,74.0,55.0,51.0,48.0,42.0,67.5
Addington West,66.0,67.0,55.0,63.0,143.0,218.0,174.0,144.0,113.0,116.0,82.0,87.0,81.0,60.0,55.5,46.5,45.0,84.0


In [48]:
# for each area calculate the proportion of each type of age range
# iterate over all areas
for index, row in age.iterrows():
  # calculate sum total
  total = row['01'] + row['02'] + row['03'] + row['04'] + row['05'] + row['06'] + \
          row['07'] + row['08'] + row['09'] + row['10'] + row['11'] + row['12'] + \
          row['13'] + row['14'] + row['15'] + row['16'] + row['17'] + row['18']
  # and calculate proportions
  age.at[index,'1'] = row['01']/total
  age.at[index,'2'] = row['02']/total
  age.at[index,'3'] = row['03']/total
  age.at[index,'4'] = row['04']/total
  age.at[index,'5'] = row['05']/total
  age.at[index,'6'] = row['06']/total
  age.at[index,'7'] = row['07']/total
  age.at[index,'8'] = row['08']/total
  age.at[index,'9'] = row['09']/total
  age.at[index,'10'] = row['10']/total
  age.at[index,'11'] = row['11']/total
  age.at[index,'12'] = row['12']/total
  age.at[index,'13'] = row['13']/total
  age.at[index,'14'] = row['14']/total
  age.at[index,'15'] = row['15']/total
  age.at[index,'16'] = row['16']/total
  age.at[index,'17'] = row['17']/total
  age.at[index,'18'] = row['18']/total
# only keep relevant columns
age = age.drop(columns=['01', '02', '03', '04', '05', '06', \
                        '07', '08', '09'])

# add name as one of the columns
age['NAME'] = age.index

# rearrange the columns
age = age[['NAME', '1', '2', '3', '4', '5', '6' ,'7', '8', '9', \
           '10', '11', '12', '13', '14', '15', '16', '17', '18']]
# display the result
age.head()

Age_group_5_year_groups_to_85_years_and_over_code,NAME,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
Area_description,,,,,,,,,,,,,,,,,,,
Abbey Caves-Glenbervie,Abbey Caves-Glenbervie,0.052632,0.056842,0.073684,0.060000,0.035789,0.027368,0.040000,0.050526,0.060000,0.094737,0.075789,0.101053,0.065263,0.076842,0.077368,0.033158,0.012632,0.006316
Abbotsford,Abbotsford,0.058695,0.066456,0.051904,0.058695,0.042687,0.068882,0.057240,0.067912,0.062091,0.061121,0.066942,0.061606,0.081494,0.058938,0.048023,0.047296,0.026922,0.013097
Acacia Bay,Acacia Bay,0.043895,0.041500,0.055866,0.044693,0.019154,0.046688,0.028731,0.046289,0.048683,0.067837,0.079808,0.079010,0.066241,0.132881,0.107741,0.056265,0.020351,0.014366
Addington East,Addington East,0.062825,0.046005,0.028197,0.039575,0.117240,0.143458,0.112293,0.074202,0.061341,0.056394,0.051447,0.040069,0.036606,0.027208,0.025229,0.023745,0.020777,0.033391
Addington West,Addington West,0.038824,0.039412,0.032353,0.037059,0.084118,0.128235,0.102353,0.084706,0.066471,0.068235,0.048235,0.051176,0.047647,0.035294,0.032647,0.027353,0.026471,0.049412


In [49]:
# merge with the rest of the data
data = pd.merge(data, age, on=["NAME", "NAME"])
data.head()

,NAME,POPESTIMATE2015,STNAME,TOT_POP,TOT_MALE,TOT_FEMLE,WHITE,HISPANIC,BLACK,ASIAN,NATIVE,OTHER,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,North Cape,1602,Northland,201141,0.498127,0.501873,0.450156,0.0,0.0,0.012461,0.526480,0.010903,0.046685,0.069094,0.084034,0.057890,0.041083,0.042951,0.044818,0.041083,0.042951,0.074697,0.075630,0.090570,0.076564,0.073763,0.061625,0.038282,0.021475,0.016807
1,Rangaunu Harbour,2310,Northland,201141,0.495449,0.504551,0.485521,0.0,0.0,0.016409,0.485521,0.012548,0.079818,0.088254,0.079169,0.067489,0.044776,0.056457,0.054510,0.041531,0.059701,0.066191,0.058404,0.075276,0.069435,0.064244,0.043478,0.022713,0.018170,0.010383
2,Karikari Peninsula,1251,Northland,201141,0.497608,0.502392,0.565302,0.0,0.0,0.011696,0.407407,0.015595,0.045346,0.064439,0.068019,0.040573,0.028640,0.028640,0.047733,0.033413,0.047733,0.064439,0.081146,0.089499,0.097852,0.118138,0.062053,0.045346,0.019093,0.017900
3,Tangonge,1134,Northland,201141,0.493369,0.506631,0.483673,0.0,0.0,0.008163,0.491837,0.016327,0.068466,0.071099,0.084266,0.057933,0.052666,0.044766,0.048716,0.051350,0.053983,0.086899,0.060566,0.086899,0.068466,0.057933,0.050033,0.030283,0.015800,0.009875
4,Ahipara,1230,Northland,201141,0.515815,0.484185,0.460259,0.0,0.0,0.014787,0.508318,0.016636,0.064204,0.084797,0.067838,0.060569,0.053301,0.050878,0.048455,0.046033,0.060569,0.059358,0.077529,0.077529,0.086008,0.071472,0.033919,0.024228,0.016959,0.016354


In [50]:
print(len(data))

2140


I have lost another 13 areas, now having a total of 2,140 areas.

## Income
The NZ census reports income differently to how Synthea uses data.<br>
I will split the categories in the following way (Synthea categories to NZ census categories:<br>
- `00..10` - `Loss`, `Zero income`, `$1-5,000`, `$5,001-10,000` (categories 11, 12, 13 and 14)
- `10..15` - `$10,001-15,000` (category 15)
- `15..25` - `$15,001-20,000` and `$20,001-25,000` (categories 16 and 17)
- `25..35` - `$25,001-30,000` and `$30,001-35,000` (categories 18 and 19)
- `35..50`- `$35,001-40,000` and `$40,001-50,000` (categories 20 and 21)
- `50..75` - `$50,001-60,000` and `$60,001-70,000` (categories 22 and 23)
- `75..100` - `$70,001-100,000` (category 24)
- `100..150` - `$100,001-150,000` (category 25)
- `150..200` - blank
- `200..999` - `$150,001 or more` (category 26)

In [51]:
# read the csv file containing income
income_source = pd.read_csv('2018-census-place-summaries-work-table5-2018-csv.csv')
income_source.head()

,Year,Area_type,Area_code,Area_description,Total_personal_income_code,Total_personal_income_description,Maori_ethnic_group_indicator_summary_code,Maori_ethnic_group_indicator_summary_description,Census_usually_resident_population_count_aged_15_years_and_over,Total_personal_income_percent,Total_personal_income_by_Maori_ethnic_group_indicator_summary_percent
0,2018,Regional Council,01,Northland Region,11,Loss,00,No 'Maori' response given,663,0.5,..
1,2018,Regional Council,01,Northland Region,11,Loss,20,Maori,324,0.2,0.7
2,2018,Regional Council,01,Northland Region,11,Loss,Total,Total,984,0.7,..
3,2018,Regional Council,01,Northland Region,12,Zero income,00,No 'Maori' response given,5112,3.6,..
4,2018,Regional Council,01,Northland Region,12,Zero income,20,Maori,3600,2.5,8.2


In [52]:
# only keep the areas
income = income_source.loc[income_source['Area_type'] == 'Statistical Area 2']
# remove totals and not stated
income = income.loc[income['Total_personal_income_description'] != 'Not stated']
income = income.loc[income['Total_personal_income_description'] != 'Total']
income = income.loc[income['Total_personal_income_description'] != 'Total stated']
# and keep totals, not split between Maori and not Maori
income = income.loc[income['Maori_ethnic_group_indicator_summary_code'] == 'Total']

# only keep relevant columns
income = income[['Area_description', 'Total_personal_income_code', 'Census_usually_resident_population_count_aged_15_years_and_over']]
# only keep numeric values - some areas have no populations
income = income[pd.to_numeric(income['Census_usually_resident_population_count_aged_15_years_and_over'], errors='coerce').notnull()]
# convert the numeric column to numeric type
income['Census_usually_resident_population_count_aged_15_years_and_over'] = pd.to_numeric(income['Census_usually_resident_population_count_aged_15_years_and_over'])
# create a pivot table
income = income.pivot_table('Census_usually_resident_population_count_aged_15_years_and_over', ['Area_description'], 'Total_personal_income_code')
# check the data
income.head()

Total_personal_income_code,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26
Area_description,,,,,,,,,,,,,,,,
Abbey Caves-Glenbervie,6.0,48.0,48.0,39.0,66.0,99.0,81.0,66.0,51.0,54.0,81.0,99.0,72.0,153.0,69.0,27.0
Abbotsford,9.0,111.0,90.0,75.0,144.0,228.0,186.0,138.0,117.0,165.0,246.0,228.0,207.0,240.0,66.0,21.0
Acacia Bay,6.0,69.0,60.0,33.0,75.0,117.0,99.0,87.0,69.0,84.0,126.0,102.0,105.0,180.0,96.0,78.0
Addington East,12.0,123.0,111.0,123.0,201.0,267.0,228.0,141.0,153.0,183.0,354.0,261.0,144.0,156.0,48.0,12.0
Addington West,3.0,159.0,120.0,87.0,141.0,231.0,189.0,123.0,114.0,147.0,276.0,228.0,129.0,129.0,45.0,9.0


In [53]:
# for each area calculate the proportion of each type of income
# iterate over all areas
for index, row in income.iterrows():
  # calculate sum total
  total = row['11'] + row['12'] + row['13'] + row['14'] + row['15'] + row['16'] + \
          row['17'] + row['18'] + row['19'] + row['20'] + row['21'] + row['22'] + \
          row['23'] + row['24'] + row['25'] + row['26']
  # and calculate proportions
  income.at[index,'00..10'] = (row['11']+row['12']+row['13']+row['14'])/total
  income.at[index,'10..15'] = row['15']/total
  income.at[index,'15..25'] = (row['16']+row['17'])/total
  income.at[index,'25..35'] = (row['18']+row['19'])/total
  income.at[index,'35..50'] = (row['20']+row['21'])/total
  income.at[index,'50..75'] = (row['22']+row['23'])/total
  income.at[index,'75..100'] = row['24']/total
  income.at[index,'100..150'] = row['25']/total
  income.at[index,'150..200'] = 0
  income.at[index,'200..999'] = row['26']/total

# add name as one of the columns
income['NAME'] = income.index

# rearrange the columns
income = income[['NAME', '00..10', '10..15', '15..25', '25..35', '35..50', '50..75', \
           '75..100', '100..150', '150..200', '200..999']]
# display the result
income.head()

Total_personal_income_code,NAME,00..10,10..15,15..25,25..35,35..50,50..75,75..100,100..150,150..200,200..999
Area_description,,,,,,,,,,,
Abbey Caves-Glenbervie,Abbey Caves-Glenbervie,0.133144,0.062323,0.169972,0.110482,0.127479,0.161473,0.144476,0.065156,0.0,0.025496
Abbotsford,Abbotsford,0.125495,0.063408,0.182299,0.112285,0.180978,0.191546,0.105680,0.029062,0.0,0.009247
Acacia Bay,Acacia Bay,0.121212,0.054113,0.155844,0.112554,0.151515,0.149351,0.129870,0.069264,0.0,0.056277
Addington East,Addington East,0.146603,0.079857,0.196663,0.116806,0.213349,0.160906,0.061979,0.019070,0.0,0.004768
Addington West,Addington West,0.173239,0.066197,0.197183,0.111268,0.198592,0.167606,0.060563,0.021127,0.0,0.004225


In [54]:
# merge with the rest of the data
data = pd.merge(data, income, on=["NAME", "NAME"])
data.head()

,NAME,POPESTIMATE2015,STNAME,TOT_POP,TOT_MALE,TOT_FEMLE,WHITE,HISPANIC,BLACK,ASIAN,NATIVE,OTHER,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,00..10,10..15,15..25,25..35,35..50,50..75,75..100,100..150,150..200,200..999
0,North Cape,1602,Northland,201141,0.498127,0.501873,0.450156,0.0,0.0,0.012461,0.526480,0.010903,0.046685,0.069094,0.084034,0.057890,0.041083,0.042951,0.044818,0.041083,0.042951,0.074697,0.075630,0.090570,0.076564,0.073763,0.061625,0.038282,0.021475,0.016807,0.179070,0.100000,0.293023,0.116279,0.125581,0.097674,0.051163,0.018605,0.0,0.018605
1,Rangaunu Harbour,2310,Northland,201141,0.495449,0.504551,0.485521,0.0,0.0,0.016409,0.485521,0.012548,0.079818,0.088254,0.079169,0.067489,0.044776,0.056457,0.054510,0.041531,0.059701,0.066191,0.058404,0.075276,0.069435,0.064244,0.043478,0.022713,0.018170,0.010383,0.173010,0.122837,0.240484,0.121107,0.152249,0.110727,0.055363,0.013841,0.0,0.010381
2,Karikari Peninsula,1251,Northland,201141,0.497608,0.502392,0.565302,0.0,0.0,0.011696,0.407407,0.015595,0.045346,0.064439,0.068019,0.040573,0.028640,0.028640,0.047733,0.033413,0.047733,0.064439,0.081146,0.089499,0.097852,0.118138,0.062053,0.045346,0.019093,0.017900,0.126471,0.135294,0.305882,0.114706,0.120588,0.120588,0.044118,0.020588,0.0,0.011765
3,Tangonge,1134,Northland,201141,0.493369,0.506631,0.483673,0.0,0.0,0.008163,0.491837,0.016327,0.068466,0.071099,0.084266,0.057933,0.052666,0.044766,0.048716,0.051350,0.053983,0.086899,0.060566,0.086899,0.068466,0.057933,0.050033,0.030283,0.015800,0.009875,0.167235,0.092150,0.221843,0.109215,0.167235,0.126280,0.078498,0.020478,0.0,0.017065
4,Ahipara,1230,Northland,201141,0.515815,0.484185,0.460259,0.0,0.0,0.014787,0.508318,0.016636,0.064204,0.084797,0.067838,0.060569,0.053301,0.050878,0.048455,0.046033,0.060569,0.059358,0.077529,0.077529,0.086008,0.071472,0.033919,0.024228,0.016959,0.016354,0.190184,0.107362,0.233129,0.110429,0.131902,0.122699,0.064417,0.015337,0.0,0.024540


In [55]:
print(len(data))

2135


I have lost another 5 areas, now having a total of 2,135 areas.

## Education
The NZ census reports education differently to how Synthea uses data.<br>
I will split the categories in the following way (Synthea categories to NZ census categories:<br>
- `LESS_THAN_HS` - categories 0, 1, 2, 3, 4 
- `HS_DEGREE` - categories 5, 6 and 11
- `SOME_COLLEGE` - none
- `BS_DEGREE` - categories 7, 8, 9 and 10

In [56]:
# read the csv file containing education
education_source = pd.read_csv('2018-census-place-summaries-education-table2-2018-csv.csv')
education_source.head()

,Year,Area_type,Area_code,Area_description,Highest_qualification_code,Highest_qualification_description,Maori_ethnic_group_indicator_summary_code,Maori_ethnic_group_indicator_summary_description,Census_usually_resident_population_count_aged_15_years_and_over,Highest_qualification_by_Maori_ethnic_group_indicator_summary_percent,Highest_qualification_percent
0,2018,Regional Council,01,Northland Region,000,No qualification,00,No 'Maori' response given,19215,..,14.7
1,2018,Regional Council,01,Northland Region,000,No qualification,20,Maori,10995,28.2,8.4
2,2018,Regional Council,01,Northland Region,000,No qualification,Total,Total,30210,..,23.1
3,2018,Regional Council,01,Northland Region,001,Level 1 certificate,00,No 'Maori' response given,13026,..,10.0
4,2018,Regional Council,01,Northland Region,001,Level 1 certificate,20,Maori,5520,14.2,4.2


In [57]:
# only keep the areas
education = education_source.loc[education_source['Area_type'] == 'Statistical Area 2']
# remove totals and not stated
education = education.loc[education['Highest_qualification_code'] != '999']
education = education.loc[education['Highest_qualification_code'] != 'Total']
education = education.loc[education['Highest_qualification_code'] != 'Total stated']
# and keep totals, not split between Maori and not Maori
education = education.loc[education['Maori_ethnic_group_indicator_summary_code'] == 'Total']

# only keep relevant columns
education = education[['Area_description', 'Highest_qualification_code', 'Census_usually_resident_population_count_aged_15_years_and_over']]
# only keep numeric values - some areas have no populations
education = education[pd.to_numeric(education['Census_usually_resident_population_count_aged_15_years_and_over'], errors='coerce').notnull()]
# convert the numeric column to numeric type
education['Census_usually_resident_population_count_aged_15_years_and_over'] = pd.to_numeric(education['Census_usually_resident_population_count_aged_15_years_and_over'])
# create a pivot table
education = education.pivot_table('Census_usually_resident_population_count_aged_15_years_and_over', ['Area_description'], 'Highest_qualification_code')
# check the data
education.head()

Highest_qualification_code,000,001,002,003,004,005,006,007,008,009,010,011
Area_description,,,,,,,,,,,,
Abbey Caves-Glenbervie,141.0,147.0,117.0,96.0,135.0,57.0,57.0,141.0,63.0,27.0,6.0,36.0
Abbotsford,525.0,336.0,282.0,225.0,330.0,102.0,84.0,189.0,54.0,21.0,12.0,39.0
Acacia Bay,198.0,192.0,177.0,108.0,150.0,81.0,90.0,183.0,69.0,39.0,3.0,45.0
Addington East,414.0,183.0,174.0,273.0,159.0,105.0,84.0,423.0,153.0,108.0,18.0,180.0
Addington West,366.0,165.0,147.0,162.0,117.0,90.0,69.0,339.0,108.0,51.0,12.0,255.0


In [58]:
# for each area calculate the proportion of each type of income
# iterate over all areas
for index, row in education.iterrows():
  # calculate sum total
  total = row['000'] + row['001'] + row['002'] + row['003'] + row['004'] + \
          row['005'] + row['006'] + row['007'] + row['008'] + row['009'] + \
          row['010'] + row['011'] 
  # and calculate proportions
  education.at[index,'LESS_THAN_HS'] = (row['001']+row['002']+row['003']+ \
                                        row['004'])/total
  education.at[index,'HS_DEGREE'] = (row['005']+row['006']+row['011'])/total
  education.at[index,'SOME_COLLEGE'] = 0
  education.at[index,'BS_DEGREE'] = (row['007']+row['008']+row['009']+ \
                                     row['010'])/total


# add name as one of the columns
education['NAME'] = education.index

# rearrange the columns
education = education[['NAME', 'LESS_THAN_HS', 'HS_DEGREE', 'SOME_COLLEGE', \
           'BS_DEGREE']]
# display the result
education.head()

Highest_qualification_code,NAME,LESS_THAN_HS,HS_DEGREE,SOME_COLLEGE,BS_DEGREE
Area_description,,,,,
Abbey Caves-Glenbervie,Abbey Caves-Glenbervie,0.483871,0.146628,0.0,0.231672
Abbotsford,Abbotsford,0.533424,0.102319,0.0,0.125512
Acacia Bay,Acacia Bay,0.469663,0.161798,0.0,0.220225
Addington East,Addington East,0.346966,0.162269,0.0,0.308707
Addington West,Addington West,0.314195,0.220096,0.0,0.271132


In [59]:
# merge with the rest of the data
data = pd.merge(data, education, on=["NAME", "NAME"])
data.head()

,NAME,POPESTIMATE2015,STNAME,TOT_POP,TOT_MALE,TOT_FEMLE,WHITE,HISPANIC,BLACK,ASIAN,NATIVE,OTHER,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,00..10,10..15,15..25,25..35,35..50,50..75,75..100,100..150,150..200,200..999,LESS_THAN_HS,HS_DEGREE,SOME_COLLEGE,BS_DEGREE
0,North Cape,1602,Northland,201141,0.498127,0.501873,0.450156,0.0,0.0,0.012461,0.526480,0.010903,0.046685,0.069094,0.084034,0.057890,0.041083,0.042951,0.044818,0.041083,0.042951,0.074697,0.075630,0.090570,0.076564,0.073763,0.061625,0.038282,0.021475,0.016807,0.179070,0.100000,0.293023,0.116279,0.125581,0.097674,0.051163,0.018605,0.0,0.018605,0.492147,0.094241,0.0,0.086387
1,Rangaunu Harbour,2310,Northland,201141,0.495449,0.504551,0.485521,0.0,0.0,0.016409,0.485521,0.012548,0.079818,0.088254,0.079169,0.067489,0.044776,0.056457,0.054510,0.041531,0.059701,0.066191,0.058404,0.075276,0.069435,0.064244,0.043478,0.022713,0.018170,0.010383,0.173010,0.122837,0.240484,0.121107,0.152249,0.110727,0.055363,0.013841,0.0,0.010381,0.518797,0.103383,0.0,0.109023
2,Karikari Peninsula,1251,Northland,201141,0.497608,0.502392,0.565302,0.0,0.0,0.011696,0.407407,0.015595,0.045346,0.064439,0.068019,0.040573,0.028640,0.028640,0.047733,0.033413,0.047733,0.064439,0.081146,0.089499,0.097852,0.118138,0.062053,0.045346,0.019093,0.017900,0.126471,0.135294,0.305882,0.114706,0.120588,0.120588,0.044118,0.020588,0.0,0.011765,0.529412,0.124183,0.0,0.104575
3,Tangonge,1134,Northland,201141,0.493369,0.506631,0.483673,0.0,0.0,0.008163,0.491837,0.016327,0.068466,0.071099,0.084266,0.057933,0.052666,0.044766,0.048716,0.051350,0.053983,0.086899,0.060566,0.086899,0.068466,0.057933,0.050033,0.030283,0.015800,0.009875,0.167235,0.092150,0.221843,0.109215,0.167235,0.126280,0.078498,0.020478,0.0,0.017065,0.514815,0.107407,0.0,0.133333
4,Ahipara,1230,Northland,201141,0.515815,0.484185,0.460259,0.0,0.0,0.014787,0.508318,0.016636,0.064204,0.084797,0.067838,0.060569,0.053301,0.050878,0.048455,0.046033,0.060569,0.059358,0.077529,0.077529,0.086008,0.071472,0.033919,0.024228,0.016959,0.016354,0.190184,0.107362,0.233129,0.110429,0.131902,0.122699,0.064417,0.015337,0.0,0.024540,0.477032,0.127208,0.0,0.183746


In [60]:
print(len(data))

2135


Still 2,135 records.

## Final dataset check and preparation

Add the missing columns: `ID`, `COUNTY` and `CTYNAME`

In [61]:
# set id based on id from the dataframe
data['ID'] = data.index

In [64]:
# Areas outside are still markes as ''
data = data.replace('','Area Outside')

In [67]:
# set a dictionary for codes for the areas
area_dict = {'Auckland': 'NZ-AUK',
             'Bay of Plenty': 'NZ-BOP',
             'Canterbury': 'NZ-CAN',
             'Gisborne': 'NZ-GIS',
             "Hawke's Bay": 'NZ-HKB',
             'Marlborough': 'NZ-MBH',
             'Manawatū-Whanganui': 'NZ-MWT',
             'Nelson': 'NZ-NSN',
             'Northland': 'NZ-NTL',
             'Otago': 'NZ-OTA',
             'Southland': 'NZ-STL',
             'Tasman': 'NZ-TAS',
             'Taranaki': 'NZ-TKI',
             'Wellington': 'NZ-WGN',
             'West Coast': 'NZ-WTC',
             'Waikato': 'NZ-WKO',
             'Area Outside': 'NZ-OUT'}

In [68]:
# for each area add the state code
# iterate over all areas
for index, row in data.iterrows():
  data.at[index,'COUNTY'] = area_dict[row['STNAME']]

In [69]:
# check that the only unique items are the codes
data['COUNTY'].unique()

array(['NZ-NTL', 'NZ-OUT', 'NZ-WKO', 'NZ-AUK', 'NZ-GIS', 'NZ-MWT',
       'NZ-CAN', 'NZ-OTA', 'NZ-BOP', 'NZ-WGN', 'NZ-HKB', 'NZ-WTC',
       'NZ-MBH', 'NZ-STL', 'NZ-TKI', 'NZ-TAS', 'NZ-NSN'], dtype=object)

In [70]:
# set CTYNAME as the same as NAME
data['CTYNAME'] = data['NAME']

In [71]:
# rearrange the columns
data = data[['ID', 'COUNTY', 'NAME', 'STNAME', 'POPESTIMATE2015', 'CTYNAME', 
             'TOT_POP', 'TOT_MALE', 'TOT_FEMLE', 'WHITE', 'HISPANIC', 'BLACK', 
             'ASIAN', 'NATIVE', 'OTHER', '1', '2', '3', '4', '5', '6', '7', '8',
             '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', 
             '00..10', '10..15', '15..25', '25..35', '35..50', '50..75', 
             '75..100', '100..150', '150..200', '200..999', 'LESS_THAN_HS',
             'HS_DEGREE', 'SOME_COLLEGE', 'BS_DEGREE']]

In [72]:
# check the data
data.head()

,ID,COUNTY,NAME,STNAME,POPESTIMATE2015,CTYNAME,TOT_POP,TOT_MALE,TOT_FEMLE,WHITE,HISPANIC,BLACK,ASIAN,NATIVE,OTHER,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,00..10,10..15,15..25,25..35,35..50,50..75,75..100,100..150,150..200,200..999,LESS_THAN_HS,HS_DEGREE,SOME_COLLEGE,BS_DEGREE
0,0,NZ-NTL,North Cape,Northland,1602,North Cape,201141,0.498127,0.501873,0.450156,0.0,0.0,0.012461,0.526480,0.010903,0.046685,0.069094,0.084034,0.057890,0.041083,0.042951,0.044818,0.041083,0.042951,0.074697,0.075630,0.090570,0.076564,0.073763,0.061625,0.038282,0.021475,0.016807,0.179070,0.100000,0.293023,0.116279,0.125581,0.097674,0.051163,0.018605,0.0,0.018605,0.492147,0.094241,0.0,0.086387
1,1,NZ-NTL,Rangaunu Harbour,Northland,2310,Rangaunu Harbour,201141,0.495449,0.504551,0.485521,0.0,0.0,0.016409,0.485521,0.012548,0.079818,0.088254,0.079169,0.067489,0.044776,0.056457,0.054510,0.041531,0.059701,0.066191,0.058404,0.075276,0.069435,0.064244,0.043478,0.022713,0.018170,0.010383,0.173010,0.122837,0.240484,0.121107,0.152249,0.110727,0.055363,0.013841,0.0,0.010381,0.518797,0.103383,0.0,0.109023
2,2,NZ-NTL,Karikari Peninsula,Northland,1251,Karikari Peninsula,201141,0.497608,0.502392,0.565302,0.0,0.0,0.011696,0.407407,0.015595,0.045346,0.064439,0.068019,0.040573,0.028640,0.028640,0.047733,0.033413,0.047733,0.064439,0.081146,0.089499,0.097852,0.118138,0.062053,0.045346,0.019093,0.017900,0.126471,0.135294,0.305882,0.114706,0.120588,0.120588,0.044118,0.020588,0.0,0.011765,0.529412,0.124183,0.0,0.104575
3,3,NZ-NTL,Tangonge,Northland,1134,Tangonge,201141,0.493369,0.506631,0.483673,0.0,0.0,0.008163,0.491837,0.016327,0.068466,0.071099,0.084266,0.057933,0.052666,0.044766,0.048716,0.051350,0.053983,0.086899,0.060566,0.086899,0.068466,0.057933,0.050033,0.030283,0.015800,0.009875,0.167235,0.092150,0.221843,0.109215,0.167235,0.126280,0.078498,0.020478,0.0,0.017065,0.514815,0.107407,0.0,0.133333
4,4,NZ-NTL,Ahipara,Northland,1230,Ahipara,201141,0.515815,0.484185,0.460259,0.0,0.0,0.014787,0.508318,0.016636,0.064204,0.084797,0.067838,0.060569,0.053301,0.050878,0.048455,0.046033,0.060569,0.059358,0.077529,0.077529,0.086008,0.071472,0.033919,0.024228,0.016959,0.016354,0.190184,0.107362,0.233129,0.110429,0.131902,0.122699,0.064417,0.015337,0.0,0.024540,0.477032,0.127208,0.0,0.183746


In [73]:
# one of the heading is still misspelled, it's TOT_FEMLE instead of TOT_FEMALE
data = data.rename(columns={'TOT_FEMLE': 'TOT_FEMALE'})
data.head()

,ID,COUNTY,NAME,STNAME,POPESTIMATE2015,CTYNAME,TOT_POP,TOT_MALE,TOT_FEMALE,WHITE,HISPANIC,BLACK,ASIAN,NATIVE,OTHER,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,00..10,10..15,15..25,25..35,35..50,50..75,75..100,100..150,150..200,200..999,LESS_THAN_HS,HS_DEGREE,SOME_COLLEGE,BS_DEGREE
0,0,NZ-NTL,North Cape,Northland,1602,North Cape,201141,0.498127,0.501873,0.450156,0.0,0.0,0.012461,0.526480,0.010903,0.046685,0.069094,0.084034,0.057890,0.041083,0.042951,0.044818,0.041083,0.042951,0.074697,0.075630,0.090570,0.076564,0.073763,0.061625,0.038282,0.021475,0.016807,0.179070,0.100000,0.293023,0.116279,0.125581,0.097674,0.051163,0.018605,0.0,0.018605,0.492147,0.094241,0.0,0.086387
1,1,NZ-NTL,Rangaunu Harbour,Northland,2310,Rangaunu Harbour,201141,0.495449,0.504551,0.485521,0.0,0.0,0.016409,0.485521,0.012548,0.079818,0.088254,0.079169,0.067489,0.044776,0.056457,0.054510,0.041531,0.059701,0.066191,0.058404,0.075276,0.069435,0.064244,0.043478,0.022713,0.018170,0.010383,0.173010,0.122837,0.240484,0.121107,0.152249,0.110727,0.055363,0.013841,0.0,0.010381,0.518797,0.103383,0.0,0.109023
2,2,NZ-NTL,Karikari Peninsula,Northland,1251,Karikari Peninsula,201141,0.497608,0.502392,0.565302,0.0,0.0,0.011696,0.407407,0.015595,0.045346,0.064439,0.068019,0.040573,0.028640,0.028640,0.047733,0.033413,0.047733,0.064439,0.081146,0.089499,0.097852,0.118138,0.062053,0.045346,0.019093,0.017900,0.126471,0.135294,0.305882,0.114706,0.120588,0.120588,0.044118,0.020588,0.0,0.011765,0.529412,0.124183,0.0,0.104575
3,3,NZ-NTL,Tangonge,Northland,1134,Tangonge,201141,0.493369,0.506631,0.483673,0.0,0.0,0.008163,0.491837,0.016327,0.068466,0.071099,0.084266,0.057933,0.052666,0.044766,0.048716,0.051350,0.053983,0.086899,0.060566,0.086899,0.068466,0.057933,0.050033,0.030283,0.015800,0.009875,0.167235,0.092150,0.221843,0.109215,0.167235,0.126280,0.078498,0.020478,0.0,0.017065,0.514815,0.107407,0.0,0.133333
4,4,NZ-NTL,Ahipara,Northland,1230,Ahipara,201141,0.515815,0.484185,0.460259,0.0,0.0,0.014787,0.508318,0.016636,0.064204,0.084797,0.067838,0.060569,0.053301,0.050878,0.048455,0.046033,0.060569,0.059358,0.077529,0.077529,0.086008,0.071472,0.033919,0.024228,0.016959,0.016354,0.190184,0.107362,0.233129,0.110429,0.131902,0.122699,0.064417,0.015337,0.0,0.024540,0.477032,0.127208,0.0,0.183746


## Save as CSV

In [74]:
#data_region.to_csv('region.csv', index=False)
#data_area.to_csv('area.csv', index=False)
data.to_csv('demographics.csv', index=False)

## Read those CSVs

In [ ]:
#data_region = pd.read_csv('region.csv')
#data_area = pd.read_csv('area.csv')
data = pd.read_csv('demographics.csv')